In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F


class SkipGramModel(nn.Module):

    def __init__(self, emb_size_u, emb_size_v,emb_dimension):
        super(SkipGramModel, self).__init__()
        self.emb_dimension = emb_dimension
        self.u_embeddings = nn.Embedding(emb_size_u, emb_dimension, sparse=False)
        self.v_embeddings = nn.Embedding(emb_size_v, emb_dimension, sparse=False)
        self.init_emb()
        

    def init_emb(self):
        initrange = 0.5 / self.emb_dimension
        self.u_embeddings.weight.data.uniform_(-initrange, initrange)
        self.v_embeddings.weight.data.uniform_(-0, 0)
        
    def forward(self, pos_u, pos_v,neg_v):
        emb_u = self.u_embeddings(pos_u)
        neg_v = neg_v.view(len(pos_u),-1)
        samples = torch.cat([pos_v,Variable(neg_v)],1)
        emb_v = self.v_embeddings(samples)
        score = torch.bmm(emb_v, emb_u.unsqueeze(2)).squeeze()
        score[:,1:]=score[:,1:].neg()
        score = F.logsigmoid(score)
        return -1 * (torch.sum(score))/ pos_u.size(0)

In [1]:
from torch.utils.data import Dataset
from collections import defaultdict
from gensim.models.doc2vec import TaggedDocument
import random

class wDataSet(Dataset):
    def __init__(self, dataset, power=0.75, neg_samples=10):
     
        #self.build_vocab(tagged_docs)
        #self.ctx_weights = torch.FloatTensor(self.make_cum_table(power))
        self.pairs = self.generate_pairs(dataset,5)
        self.neg_samples=neg_samples
        
    def generate_pairs(self, dataset, ctx_window):
            pairs = []
            for sentence in dataset:
                for i,word in enumerate(sentence):
                    for j in range(1,ctx_window):
                        pairs.append((word,sentece[i+j]))
                        if((i-j)>0):
                            pairs.append((word,sentence[i-j]))
                            
            return pairs
                            
        
    def __len__(self):
        return 5
        
    def __getitem__(self, idx):
        return 0




In [58]:
from gensim.test.utils import datapath
import gensim.downloader as api
#sentences = LineSentence(datapath('lee_background.cor'))
dataset = api.load('text8')
print(type(dataset))
p = []
for x in dataset: 
    p.append(x)
    
"""
print(len(p))
sum = 0
for x in p: 
    sum += len(x)
print(sum/len(p))
print(len(p[0]))
"""

<class 'text8.Dataset'>


'\nprint(len(p))\nsum = 0\nfor x in p: \n    sum += len(x)\nprint(sum/len(p))\nprint(len(p[0]))\n'

In [61]:
dataset = wDataSet(p)

AttributeError: 'wDataSet' object has no attribute 'generate_pairs'